In [0]:
## please note that this webscraper most likely won't work, \
## thecarconnection.com has noticed the use of this webscraper and did not \
## make use of different URLs for different trims anymore. i will attempt to \
## update the code shortly, at a significant loss, because it will reduce \
## the number of cars by a factor of 10-15. sorry for the inconvenience

# import all the libraries useful for webscraping

# work with beautiful soup
import bs4 as bs
from urllib.request import Request, urlopen
import pandas as pd

# import on the decided website - carconnection (as of Nov)
website = "https://www.thecarconnection.com"

# define a list of all new cars - obtained from the home page of the car connection
# use the fetch function and save the list
def new_cars():
    new_cars_list = []
    for a in fetch(website, "/new-cars").find_all("a", {"class": "add-zip"}):
        new_cars_list.append(a['href'])
    return new_cars_list

# define a list to save from the year_model overview
def year_model_():
    year_model_list = []
    for make in model_menu():
        for id in fetch(website, make).find_all("a", {"id": "ymm-nav-specs-btn"}):
            year_model_().append(id['href'])
    year_model_list.remove("/specifications/buick_enclave_2019_fwd-4dr-preferred")
    return year_model_list

# trim function is from now on
def trims():
    trim_list = []
    for row in year_model_():
        div = fetch(website, row).find_all("div", {"class": "block-inner"})[-1]
        div_a = div.find_all("a")
        for i in range(len(div_a)):
            trim_list.append(div_a[-i]['href'])
    return trim_list

# make the menu function for 
def menu_fn1():
    menu_fn_list = []
    for make in new_cars():
        for div in fetch(website, make).find_all("div", {"class": "name"}):
            menu_fn_list.append(div.find_all("a")[0]['href'])
    return menu_fn_list

# menu function 2
def menu_fn2():
    menu_fn2_list = []
    for make in menu_fn1():
        soup = fetch(website, make)
        for div in soup.find_all("a", {"class": "btn avail-now first-item"}):
            menu_fn2_list.append(div['href'])
        for div in soup.find_all("a", {"class": "btn 1"}): 
            menu_fn2_list.append(div['href'])
    return menu_fn2_list

# fetch the URL 
def fetch(hostname, filename):
    return bs.BeautifulSoup(urlopen(Request(hostname + filename, headers={'User-Agent': 'X'})).read(), 'lxml')

#pd.DataFrame(trims()).to_csv(<REDACTED>, index=False, header=None)
#trims = pd.read_csv(<REDACTED>)

# the function for specifications
def specs():
    specs_table = pd.DataFrame()
    for row in trims.iloc[:, 0]:
        soup = fetch(website, row)
        specs_df = pd.DataFrame(columns=[soup.find_all("title")[0].text[:-15]])
        msrp_text = soup.find_all("div", {"class": "price"})[0]
        # length of the msrp will be found in this if loop
        if len(msrp_text.find_all("a")) >= 1:
            specs_df.loc["MSRP"] = msrp_text.find_all("a")[0].text
        # find all - specifications
        for div in soup.find_all("div", {"class": "specs-set-item"}):
            row_name = div.find_all("span")[0].text
            row_value = div.find_all("span")[1].text
            specs_df.loc[row_name] = row_value
        # add in the specifications tables
        specs_table = pd.concat([specs_table, specs_df], axis=1, sort=False)
    return specs_table

#specs().to_csv(<REDACTED>)
specs

<function __main__.specs>